In [18]:
import re
import math
import warnings
import tqdm
import json
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import optimize


warnings.filterwarnings("ignore")

In [19]:
data1 = pd.read_excel('2c治疗方案.xlsx') # 治疗方案
data2 = pd.read_excel('2a数据_已替换时间戳_已矫正.xlsx') # 水肿数据

In [20]:
data3 = pd.merge(data1, data2, how='outer', on='ID')
data3 = data3.drop(range(101, 131))
data3

,ID,脑室引流,止血治疗,降颅压治疗,降压治疗,镇静、镇痛治疗,止吐护胃,营养神经,发病到首次影像检查时间间隔,入院首次影像检查流水号,...,随访4流水号,ED_volume.4,随访5流水号,ED_volume.5,随访6流水号,ED_volume.6,随访7流水号,ED_volume.7,随访8流水号,ED_volume.8
0,sub001,0,1,1,1,1,1,1,2.5,9000,...,1531938.0,126558.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,sub002,0,1,1,1,0,1,1,3.0,10800,...,4029091.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,sub003,0,1,1,1,1,1,1,2.0,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,sub004,0,1,1,1,0,0,0,1.0,3600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,sub005,0,1,1,0,0,1,1,5.0,18000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,sub156,0,1,1,1,0,1,1,5.5,19800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156,sub157,1,0,1,1,1,1,1,0.5,1800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
157,sub158,0,1,1,1,1,1,1,2.8,10080,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
158,sub159,0,1,1,1,1,1,1,3.0,10800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
import json
with open('best_labels.json','r') as f:
    groups_label=json.load(f)
print(len(groups_label['best_labels']))

100


In [23]:
dataT=data3.head(100)
dataT['分组信息']=groups_label
dataT

,ID,脑室引流,止血治疗,降颅压治疗,降压治疗,镇静、镇痛治疗,止吐护胃,营养神经,发病到首次影像检查时间间隔,入院首次影像检查流水号,...,ED_volume.4,随访5流水号,ED_volume.5,随访6流水号,ED_volume.6,随访7流水号,ED_volume.7,随访8流水号,ED_volume.8,分组信息
0,sub001,0,1,1,1,1,1,1,2.5,9000,...,126558.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,sub002,0,1,1,1,0,1,1,3.0,10800,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,sub003,0,1,1,1,1,1,1,2.0,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,sub004,0,1,1,1,0,0,0,1.0,3600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,sub005,0,1,1,0,0,1,1,5.0,18000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,sub096,0,0,0,1,1,1,1,1.0,3600,...,231.0,2144314.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,sub097,0,1,1,1,1,1,1,3.0,10800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,sub098,0,1,1,1,1,1,1,2.5,9000,...,17288.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,sub099,1,1,1,1,1,1,1,3.0,10800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
